# 유튜버 크롤링 후 DB 적재

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import numpy as np
import psycopg2
import re

In [2]:
# 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

# 해시태그 함수
def get_hashtag(outline_test) :
    # hash_tag가 안달려있는 영상이 있기때문에 오류 방지
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        print(hash_tag)
        hashtag_list.append(hash_tag)
            
    except Exception as e:
        hash_tag = ""
        print(hash_tag)
        hashtag_list.append(hash_tag)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

In [55]:
#업로더 검색
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

#업로더 홈페이지 접속
url = 'https://www.youtube.com/channel/UC_XR-igVnkqf2A3lEpye-mQ/videos'
driver.get(url)

driver.maximize_window()

멜로우, 배배자매, 깡스타일리스트, 안녕하새은, 핏더사이즈, 짱구대디, 혜인, 송이송이, 미니월드, samedifference, 오정규

#카테고리 리스트
keyword_list = ['조명', '무드등', '형광등', '방등', '거실등', \
    '침구', '매트리스', '블라인드', '커튼', '러그', '매트', '쿠션', '소파', \
    '테이블', '식탁', '책상', '거실장', '서랍', '진열장', '책장', '선반', '행거', '옷장', '의자', '화장대', '거울']

한주의책이야기, 유튜북진서, 한뼘서재, 따뜻한책한잔오디오북, 책읽기좋은날, 그남자의목소리

In [56]:
# 제목 & 링크 크롤링

# 스크롤 다운 20번 실행
body = driver.find_element_by_tag_name('body')
body.send_keys(Keys.PAGE_DOWN)

for i in range(1,20):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

soup = BeautifulSoup(driver.page_source, 'html.parser')
name = soup.select('a#video-title')
video_url = soup.select('a#video-title')

video_num_list = [] #영상번호
name_list = [] #영상제목
url_list = [] #영상주소

for i in range(len(name)):
    video_num_list.append(i+1)
    name_list.append(name[i].text.strip())

for i in video_url:
    url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))

print('영상 개수: ', len(name_list))

영상 개수:  119


In [57]:
# 업로드일 & 해시태그 가져오기
day_list = [] #업로드일
hashtag_list = [] #해시태그

for i in range(0,len(name_list)):
    
    if url_list[i].split('/')[3] == 'shorts':  #쇼츠

        print(i+1, ': 쇼츠', end=' ')

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        driver.get(url_list[i])

        time.sleep(2)

        #더보기 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
        time.sleep(2)

        #설명 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
        time.sleep(2)

        #업로드일 정보 가져오기
        day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()

        print(day)
        day_list.append(day)

        time.sleep(2) 

        # 해시태그 가져오기
        outline_test = driver.find_element_by_class_name("style-scope ytd-reel-description-sheet-renderer").text
        get_hashtag(outline_test)

    else :
        print(i+1, ': 일반', end=' ')

        driver.get(url_list[i])
            
        time.sleep(2)    

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        time.sleep(2)  

        #업로드일 정보 가져오기
        #day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
        day = driver.find_element_by_id("info-strings").text.split('\n')[0]
        
        print(day)
        day_list.append(day)

        time.sleep(2) 

        # 해시태그 가져오기
        outline_test = soup.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
        get_hashtag(outline_test)

print('끝')

1 : 일반 2022. 6. 8.
#달세뇨 #오디오북
2 : 일반 2022. 6. 1.
#달세뇨 #오디오북
3 : 일반 2022. 5. 27.
#달세뇨 #오디오북
4 : 일반 2022. 5. 25.
#달세뇨 #오디오북
5 : 일반 2022. 5. 18.
#달세뇨 #오디오북
6 : 일반 2022. 5. 11.
#달세뇨 #오디오북
7 : 일반 2022. 5. 4.
#달세뇨 #오디오북
8 : 일반 2022. 4. 27.
#달세뇨 #오디오북
9 : 일반 2022. 4. 22.
#달세뇨 #오디오북
10 : 일반 2022. 4. 20.
#달세뇨 #오디오북
11 : 일반 2022. 4. 15.
#달세뇨 #오디오북
12 : 일반 2022. 4. 13.
#달세뇨 #오디오북
13 : 일반 2022. 4. 6.
#달세뇨 #오디오북
14 : 일반 2022. 3. 30.
#달세요 #오디오북
15 : 쇼츠 2022. 3. 24.
#나의라임오렌지나무 #오디오북
16 : 일반 2022. 3. 23.
#달세뇨 #오디오북
17 : 일반 2022. 3. 16.
#달세뇨 #오디오북
18 : 일반 2022. 3. 11.
#달세뇨 #오디오북
19 : 쇼츠 2022. 3. 10.
#빨간머리앤 #오디오북
20 : 일반 2022. 3. 8.
#달세뇨 #오디오북
21 : 일반 2022. 3. 2.
#달세뇨 #오디오북
22 : 쇼츠 2022. 2. 26.
#셜록홈즈 #노우드건축업자
23 : 쇼츠 2022. 2. 24.
#이솝우화 #여우
24 : 일반 2022. 2. 23.
#달세뇨 #오디오북
25 : 일반 2022. 2. 16.
#달세뇨 #오디오북
26 : 일반 2022. 2. 9.
#달세뇨 #오디오북
27 : 일반 2022. 2. 2.
#달세뇨 #오디오북
28 : 일반 2022. 1. 26.
#달세뇨 #오디오북
29 : 일반 2022. 1. 19.
#달세뇨 #오디오북
30 : 일반 2022. 1. 12.
#달세뇨 #오디오북
31 : 일반 2022. 1. 5.
#달세뇨 #오디오북
32 : 일반 2021. 1

In [58]:
day_list2 = [] #전처리 후 업로드일

#업로드일 날짜형식으로 통일화
for day in day_list : 
    day = day.replace('.', '')
    day = day.replace('최초 공개: ', '')
    day = day.replace('실시간 스트리밍 시작일: ', '')
    day = day.replace(' ', '-')
    day_list2.append(day)

#df화
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list2,
    '주소': url_list,
    '해시태그' : hashtag_list
}

youtubeDf = pd.DataFrame(youtubeDic)

# 업로드한 지 1년 이상된 데이터는 제거함
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])
youtubeDf = youtubeDf[youtubeDf['업로드일'] > '2021-06-01']

youtubeDf.tail()

,제목,업로드일,주소,해시태그
66,자면서 듣는 이야기 / 이무영의 제 1과 제 1장 / ASMR,2021-06-30,https://www.youtube.com/watch?v=B6gGux0FgEM,#달세뇨 #오디오북 #제
67,디즈니의 악당들 시리즈 여섯번째 이야기 / 개를 훔친 이웃집 여자,2021-06-29,https://www.youtube.com/watch?v=EQEbR0BGNmU,#달세뇨 #오디오북 #라곰
68,자면서 듣는 세계명작소설 / 톨스토이의 이반일리치의 죽음 / ASMR,2021-06-23,https://www.youtube.com/watch?v=ZQpAjw9ZRbc,#달세뇨 #톨스토이 #이반일리치
69,달세뇨가 읽어주는 BGM없는 데미안 | 전 연령대 베스트셀러 1위 |ASMR,2021-06-16,https://www.youtube.com/watch?v=e6wIz0m2mVU,#달세뇨 #오디오북 #데미안
70,달세뇨가 읽어주는 춘향전/ASMR,2021-06-02,https://www.youtube.com/watch?v=I4-EEifScLU,#달세뇨 #춘향전


In [59]:
#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_books'

#table에 데이터 넣기
for i in range(len(youtubeDf)) :
    cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
    conn.commit()

#db 연결 종료
conn.cursor().close()
conn.close()

# #############여기까지만#############

In [16]:
youtubeDf.to_csv('C:\jupiter_workspace\zam_project\outputs/유튜브_'+keyword + '.csv', encoding='utf-8-sig', index=False)

In [12]:
# 댓글 크롤링
video_num = 0
video_idex=[]
video_comm = []


for i in range(len(name_list)):
    driver.get(url_list[i])
    
    time.sleep(3)
    
    try :
        # 동영상 재생 멈추기 
        body = driver.find_element_by_tag_name('body')
        body.send_keys(Keys.SPACE)
    except: # 스페이스 누르기 안되면 직접 일시중지 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[1]/div/div/div/ytd-player/div/div/div[1]/video').click()
        
    # 유튜브 프리미엄 팝업창 닫기
    try:
        driver.find_element_by_css_selector("#dismiss-button > a").click()
    except:
        pass

    # 댓글 보이게 하기(밑으로 내리기)
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,800);")
    time.sleep(8)
    
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(3)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(5)
    
        
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    comment_list = soup.select("yt-formatted-string#content-text")

    comment_final = []

    for i in range(len(comment_list)):
        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        temp_comment = temp_comment.replace('\xa0@', '')
        temp_comment = temp_comment.replace('\xa0', '')
        temp_comment = temp_comment.replace('\r', '')
        comment_final.append(temp_comment)
    
    video_idex.append(video_num)
    video_comm.append(comment_final)
    video_num += 1


KeyboardInterrupt: 

In [23]:
df = []
for i in range(len(video_idex)):
    if len(video_comm[i]) != 0:
        n = np.full((len(video_comm[i])),i+1)
        temp = pd.DataFrame([ x for x in zip(n,video_comm[i])])
        df.append(temp)

In [24]:
comment_df = df[0]
for i in range(1, len(df)):
    comment_df = pd.concat([comment_df, df[i]])
    
comment_df.columns = ['영상 번호', '댓글']
comment_df.head()

,영상 번호,댓글
0,1,이런 컨텐츠 좋은 거 같아요~몰랐던 브래드도 알고 메인템도 알아서 좋네요~
1,1,각 브랜드에서 매력있는 아이템들이 많은걸 알아서 좋았네요 ㅎㅎ
2,1,형 이런 컨셉 유료광고면 한달에 3번 이상 유료광고 받아도 볼만한 느낌… 짱이다 형️
3,1,마지막 바지 예뻐서 사려고봤더니 품절이네요 ㅠㅠ 재입고되면 바로사야지 !
4,1,와 데님셔츠는 가격대비 미치긴한듯


In [11]:
#CSV형태로 저장하기
youtubeDf.to_csv('./'+'유튜브_핏더사이즈_크롤링.csv', encoding='utf-8-sig', index=False)
comment_df.to_csv('./'+'유튜브_핏더사이즈_댓글.csv', encoding='utf-8-sig', index=False)